# Run Experiments

## Imports

In [1]:
# Disables generation of pycache file
import sys
sys.dont_write_bytecode = True

# Imports libraries
import time
import tracemalloc
import os
import pandas as pd

# Imports made functions
from solvers import *
from utility import *

## Running Experiments

In [2]:
result_columns_basenames = ["||H||_1" , "||H||_0", "r(H)", "||AHA - A||_F", "||HAH - H||_F",
                "||(HA)^T - HA||_F", "||AHb - b||_1", "|lambda^T(AHb - b)|",
                "Computational_Time(s)", "Memory_Used(MiB)"]

problems = ["1_norm_PN", "1_norm_P1_P4", "1_norm_MSN"]

result_column_names = []
for problem in problems:
    for basename in result_columns_basenames:
        result_column_names.append(f"{problem}_{basename}")

column_names = ["m", "r"] + result_column_names

m_values = [20 + 20*i for i in range(0, 3)]

csv_file_path = "results_3.csv"
temp_csv_file_path = "results_3_temp.csv"
excel_file_path = "results_3.xlsx"

In [3]:
solvers = {
    "1_norm_PN": problem_1_norm_PN_solver,
    "1_norm_P1_P4": problem_1_norm_P1_P4_solver,
    "1_norm_MSN": problem_1_norm_MSN_solver
}

is_viable_checks = {
    "1_norm_PN": problem_1_norm_PN_viable_solution,
    "1_norm_P1_P4": problem_1_norm_P1_P4_viable_solution,
    "1_norm_MSN": problem_1_norm_MSN_viable_solution
}

In [4]:
try:
    df = pd.read_csv(csv_file_path)
except:
    # Creates an empty dataframe with the specified column names
    df = pd.DataFrame(columns=column_names)

    # Saves dataframe as a csv file
    df.to_csv(csv_file_path, index=False)

In [5]:
df = pd.read_csv(csv_file_path)

In [6]:
for m in m_values:
    A, b = generate_random_rank_r_square_matrix_vector_b(m=m)
    r = int(np.floor(0.5 * m))
    instance_results = {
        "m": m,
        "r": r
    }

    for i in range(len(problems)):
        problem = problems[i]
        solver = solvers[problem]
        is_viable_check = is_viable_checks[problem]
        # Starts tracing memory usage
        tracemalloc.start()
        start_time = time.time()
        H_star = solver(A=A, b=b)
        end_time = time.time()
        # Gets current amount of memory in usage and peak memory usage since tracing
        current, peak = tracemalloc.get_traced_memory() # Memory usage is in bytes
        # Resets peak memory usage
        tracemalloc.reset_peak()
        lambda_values = problem_2_norm_KKT_solver(A=A, b=b)
        if (problem != "1_norm_PN"):
            if (not is_viable_check(A=A, H=H_star, m=m)):
                print(f"m: {m}")
                print(f"problem: {problem} did not find a viable solution")
                sys.exit()
        else:
            if (not is_viable_check(A=A, H=H_star, b=b, lambda_values=lambda_values, m=m)):
                print(f"m: {m}")
                print(f"problem: {problem} did not find a viable solution")
                sys.exit()
        problem_results = calculate_problem_results_3(
            A=A, H=H_star, b=b, lambda_values=lambda_values, problem=problem
        )
        for key, value in problem_results.items():
            instance_results[key] = value
        instance_results[f"{problem}_Computational_Time(s)"] = end_time - start_time
        instance_results[f"{problem}_Memory_Used(MiB)"] = peak / (1024 ** 2)

    df.loc[len(df)] = instance_results
    df.to_csv(csv_file_path, index=False)

In [9]:
# Creates new csv file with different column name
with open(csv_file_path, "r") as file:
    data = file.readlines()
    for i in range(len(data)):
        data[i] = data[i].replace(f"{problems[0]}_", "")
        data[i] = data[i].replace(f"{problems[1]}_", "")
        data[i] = data[i].replace(f"{problems[2]}_", "")
    with open(temp_csv_file_path, "w")as file2:
        for line in data:
            file2.write(line)

import csv
from openpyxl import Workbook

# Creates a new Excel sheet
workbook = Workbook()
sheet = workbook.active

# Reads and csv file and write the data on the Excel sheet
with open(temp_csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        sheet.append(row)

# Saves the Excel file
workbook.save(excel_file_path)

# Deletes temp csv file
# Verifies if file exists before deleting
if os.path.exists(temp_csv_file_path):
    os.remove(temp_csv_file_path)